In [10]:
# Import all of the things you need to import!
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer

# Homework 14 (or so): TF-IDF text analysis and clustering

Hooray, we kind of figured out how text analysis works! Some of it is still magic, but at least the **TF** and **IDF** parts make a little sense. Kind of. Somewhat.

No, just kidding, we're *professionals* now.

## Investigating the Congressional Record

The [Congressional Record](https://en.wikipedia.org/wiki/Congressional_Record) is more or less what happened in Congress every single day. Speeches and all that. A good large source of text data, maybe?

Let's pretend it's totally secret but we just got it leaked to us in a data dump, and we need to check it out. It was leaked from [this page here](http://www.cs.cornell.edu/home/llee/data/convote.html).

In [1]:
# If you'd like to download it through the command line...
!curl -O http://www.cs.cornell.edu/home/llee/data/convote/convote_v1.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9607k  100 9607k    0     0   693k      0  0:00:13  0:00:13 --:--:--  407k


In [2]:
# And then extract it through the command line...
!tar -zxf convote_v1.1.tar.gz

You can explore the files if you'd like, but we're going to get the ones from `convote_v1.1/data_stage_one/development_set/`. It's a bunch of text files.

In [3]:
# glob finds files matching a certain filename pattern
import glob

# Give me all the text files
paths = glob.glob('convote_v1.1/data_stage_one/development_set/*')
paths[:5]

['convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327025_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327044_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327046_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_1479036_DON.txt']

In [4]:
len(paths)

702

So great, we have 702 of them. Now let's import them.

In [7]:
speeches = []
for path in paths:
    with open(path) as speech_file:
        speech = {
            'pathname': path,
            'filename': path.split('/')[-1],
            'content': speech_file.read()
        }
    speeches.append(speech)
speeches_df = pd.DataFrame(speeches)
speeches_df.head()

,content,filename,pathname
0,"mr. chairman , i thank the gentlewoman for yie...",052_400011_0327014_DON.txt,convote_v1.1/data_stage_one/development_set/05...
1,"mr. chairman , i want to thank my good friend ...",052_400011_0327025_DON.txt,convote_v1.1/data_stage_one/development_set/05...
2,"mr. chairman , i rise to make two fundamental ...",052_400011_0327044_DON.txt,convote_v1.1/data_stage_one/development_set/05...
3,"mr. chairman , reclaiming my time , let me mak...",052_400011_0327046_DON.txt,convote_v1.1/data_stage_one/development_set/05...
4,"mr. chairman , i thank my distinguished collea...",052_400011_1479036_DON.txt,convote_v1.1/data_stage_one/development_set/05...


In class we had the `texts` variable. For the homework can just do `speeches_df['content']` to get the same sort of list of stuff.

**Take a look at the contents of the first 5 speeches**

In [16]:
for i in speeches_df['content'].head(5):
    print(i)

mr. chairman , i thank the gentlewoman for yielding me this time . 
my good colleague from california raised the exact and critical point . 
the question is , what happens during those 45 days ? 
we will need to support elections . 
there is not a single member of this house who has not supported some form of general election , a special election , to replace the members at some point . 
but during that 45 days , what happens ? 
the chair of the constitution subcommittee says this is what happens : martial law . 
we do not know who would fill the vacancy of the presidency , but we do know that the succession act most likely suggests it would be an unelected person . 
the sponsors of the bill before us today insist , and i think rightfully so , on the importance of elections . 
but to then say that during a 45-day period we would have none of the checks and balances so fundamental to our constitution , none of the separation of powers , and that the presidency would be filled by an unel

# Doing our analysis

Use the `sklearn` package and a plain boring `CountVectorizer` to get a list of all of the tokens used in the speeches. If it won't list them all, that's ok! Make a dataframe with those terms as columns.

**Be sure to include English-language stopwords**

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(speeches_df['content'])
# print(count_vectorizer.get_feature_names())
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

,000,00007,018,050,092,10,100,106,107,108,...,yours,youth,yuan,zero,zeroing,zeros,zigler,zirkin,zoe,zoellick
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

In [82]:
def boring_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

Okay, it's **far** too big to even look at. Let's try to get a list of features from a new `CountVectorizer` that only takes the top 100 words.

In [96]:
count_vectorizer2 = CountVectorizer(stop_words='english',max_features=100,tokenizer=boring_tokenizer)
X = count_vectorizer2.fit_transform(speeches_df['content'])
# print(count_vectorizer.get_feature_names())

Now let's push all of that into a dataframe with nicely named columns.

In [84]:
pd.DataFrame(X.toarray(), columns=count_vectorizer2.get_feature_names())

,1,2,act,allow,amend,american,amp,ani,appropri,associ,...,urg,use,veri,vote,wa,want,way,work,year,yield
0,0,0,3,1,2,3,0,0,0,0,...,0,0,2,1,1,1,2,0,0,2
1,0,0,1,1,1,0,0,0,0,0,...,0,0,1,1,0,1,3,0,1,0
2,0,0,1,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,0,0
4,0,0,0,0,1,0,0,0,1,0,...,0,0,1,2,0,0,0,0,0,2
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,3,0,0,1,0,...,0,0,0,2,0,0,0,0,0,0


Everyone seems to start their speeches with "mr chairman" - how many speeches are there total, and many don't mention "chairman" and how many mention neither "mr" nor "chairman"?

In [85]:
df = pd.DataFrame(X.toarray(), columns=count_vectorizer2.get_feature_names())
mc = pd.DataFrame([df['mr'], df['chairman'], df['mr'] + df['chairman']], index=["mr", "chairman", "mr + chairman"]).T

count_mc = 0
for index, row in mc.iterrows():
    if row['mr'] != 0 and row['chairman'] != 0:
        count_mc += 1
print(count_mc,'speeches start with "mr chairman".')

count_nc = 0
for i in mc['chairman']:
    if i == 0:
        count_nc += 1
print(count_nc,"speeches don't mention chairman.")

count_nmc = 0
for i in mc['mr + chairman']:
    if i == 0:
        count_nmc += 1
print(count_nmc,'speeches mention neither "mr" nor "chairman".')

449 speeches start with "mr chairman".
250 speeches don't mention chairman.
75 speeches mention neither "mr" nor "chairman".


What is the index of the speech thank is the most thankful, a.k.a. includes the word 'thank' the most times?

In [87]:
df['thank'].max()

9

In [90]:
df[df['thank']==9].index

Int64Index([577], dtype='int64')

If I'm searching for `China` and `trade`, what are the top 3 speeches to read according to the `CountVectoriser`?

In [118]:
count_vectorizer3 = CountVectorizer(stop_words='english',tokenizer=boring_tokenizer)
X3 = count_vectorizer3.fit_transform(speeches_df['content'])
# print(count_vectorizer.get_feature_names())
ct = pd.DataFrame(X3.toarray(), columns=count_vectorizer3.get_feature_names())
print("If I'm searching for China and trade, I should look at:")
ctarray = pd.DataFrame([df['china'], df['trade'], df['china'] + df['trade']], index=["china", "trade", "china + trade"]).T
ctarray.sort_values(by="china + trade",ascending=False).head(3)

If I'm searching for China and trade, I should look at:


,china,trade,china + trade
379,28,64,92
399,27,10,37
367,5,22,27


Now what if I'm using a `TfidfVectorizer`?

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [124]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=boring_tokenizer, use_idf=False, norm='l1')
X4 = tfidf_vectorizer.fit_transform(speeches_df['content'])
tf_pd = pd.DataFrame(X4.toarray(), columns=tfidf_vectorizer.get_feature_names())
ctarray = pd.DataFrame([tf_pd['china'], tf_pd['trade'], tf_pd['china'] + tf_pd['trade']], index=["china", "trade", "china + trade"]).T
ctarray.sort_values(by="china + trade",ascending=False).head(3)

,china,trade,china + trade
345,0.076555,0.052632,0.129187
329,0.000000,0.111111,0.111111
402,0.049505,0.059406,0.108911


**What's the content of the speeches?** Here's a way to get them:

In [125]:
# index 0 is the first speech, which was the first one imported.
paths[0]

'convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt'

In [149]:
# Pass that into 'cat' using { } which lets you put variables in shell commands
# that way you can pass the path to cat
!cat {paths[345]}

i thank the gentleman from new york for yielding me this time . 
madam speaker , we have been here before . 
congress has often resorted to bills and memoranda of understanding concerning china . 
but the u.s. trade deficit with china has continued to increase . 
so i am not going to stand here and argue process . 
we can look at the history and the fact of the whole architecture of agreements that we have had with china , memoranda of understanding , concerns that members of congress from both sides of the aisle brought to this floor in order to try to manage united states trade with china . 
remember we were told that a memorandum of understanding on prison labor with china would remove their competitive advantage and restore balanced trade . 
but the u.s. trade deficit with china worsened . 
remember the agreement to reaffirm the 1992 market access memorandum of understanding . 
we passed that , but the u.s. trade deficit with china grew worse . 
remember china 's agreement to lower

**Now search for something else!** Another two terms that might show up. `elections` and `chaos`? Whatever you thnik might be interesting.

In [150]:
vyarray = pd.DataFrame([tf_pd['american'], tf_pd['vote'], tf_pd['america'] + tf_pd['vote']], index=["american", "vote", "american + vote"]).T
vyarray.sort_values(by="american + vote",ascending=False).head(3)

,american,vote,american + vote
245,0.0,0.25,0.25
701,0.0,0.20,0.20
216,0.0,0.20,0.20


In [151]:
!cat {paths[245]}

mr. chairman , no rollcall vote . 


# Enough of this garbage, let's cluster

Using a **simple counting vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency inverse document frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

In [158]:
vectorizer = CountVectorizer(tokenizer=boring_tokenizer, stop_words='english')
X = vectorizer.fit_transform(speeches_df['content'])

from sklearn.cluster import KMeans

number_of_clusters = 8
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)
print("Simple counting vectorizer:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Simple counting vectorizer,:
Cluster 0: thi mr amend s time
Cluster 1: head start program right religi
Cluster 2: amp nbsp p gt lt
Cluster 3: mr thi chairman yield gentleman
Cluster 4: rule 11 state feder court
Cluster 5: start head program children thi
Cluster 6: associ nation restaur contractor chamber
Cluster 7: thi wa s elect church


In [159]:
vectorizer = TfidfVectorizer(use_idf=True, tokenizer=boring_tokenizer, stop_words='english')
X = vectorizer.fit_transform(speeches_df['content'])

from sklearn.cluster import KMeans

number_of_clusters = 8
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)
print("Term frequency vectorizer:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Term frequency vectorizer:
Cluster 0: amend consent claim opposit unanim
Cluster 1: balanc time chairman mr reserv
Cluster 2: yield gentleman mr minut chairman
Cluster 3: hors wild land thi blm
Cluster 4: thi mr amend chairman time
Cluster 5: china trade thi s currenc
Cluster 6: demand record vote mr chairman
Cluster 7: start head program children thi


In [161]:
vectorizer = TfidfVectorizer(use_idf=True, tokenizer=boring_tokenizer, stop_words='english')
X = vectorizer.fit_transform(speeches_df['content'])

from sklearn.cluster import KMeans

number_of_clusters = 8
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)
print("Term frequency inverse document frequency vectorizer,:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Term frequency inverse document frequency vectorizer,:
Cluster 0: china trade thi s enforc
Cluster 1: gentleman yield mr chairman texa
Cluster 2: demand record vote mr chairman
Cluster 3: thi mr amend chairman time
Cluster 4: balanc time chairman mr reserv
Cluster 5: start head program children thi
Cluster 6: minut mr yield gentleman 2
Cluster 7: lawsuit court frivol thi rule


**Which one do you think works the best?**

# Harry Potter time

I have a scraped collection of Harry Potter fanfiction at https://github.com/ledeprogram/courses/raw/master/algorithms/data/hp.zip.

I want you to read them in, vectorize them and cluster them. Use this process to find out **the two types of Harry Potter fanfiction**. What is your hypothesis?

In [169]:
!curl -LO https://github.com/ledeprogram/courses/raw/master/algorithms/data/hp.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   149  100   149    0     0    660      0 --:--:-- --:--:-- --:--:--   662
100 9226k  100 9226k    0     0   812k      0  0:00:11  0:00:11 --:--:-- 1221k


In [171]:
!unzip hp.zip

Archive:  hp.zip
   creating: hp/
  inflating: hp/.DS_Store            
   creating: __MACOSX/
   creating: __MACOSX/hp/
  inflating: __MACOSX/hp/._.DS_Store  
  inflating: hp/10001898.txt         
  inflating: hp/10004131.txt         
  inflating: hp/10004927.txt         
  inflating: hp/10007980.txt         
  inflating: hp/10010343.txt         
  inflating: hp/10017757.txt         
  inflating: hp/10018490.txt         
  inflating: hp/10018889.txt         
  inflating: hp/10019142.txt         
  inflating: hp/10019987.txt         
  inflating: hp/10021604.txt         
  inflating: hp/10023278.txt         
  inflating: hp/10023376.txt         
  inflating: hp/10026221.txt         
  inflating: hp/10035136.txt         
  inflating: hp/10036003.txt         
  inflating: hp/10037071.txt         
  inflating: hp/10038079.txt         
  inflating: hp/10038493.txt         
  inflating: hp/10041730.txt         
  inflating: hp/10043489.txt         
  inflating: hp/10043782.txt         
  in

In [177]:
paths = glob.glob('hp/*')
paths[:5]

['hp/10001898.txt',
 'hp/10004131.txt',
 'hp/10004927.txt',
 'hp/10007980.txt',
 'hp/10010343.txt']

In [176]:
len(paths)

1874

In [186]:
fanfics = []
for path in paths:
    with open(path) as hp_file:
        fanfic = {
            'pathname': path,
            'filename': path.split('/')[-1],
            'content': hp_file.read()
        }
    fanfics.append(fanfic)
fanfics_df = pd.DataFrame(fanfics)
fanfics_df.head()

,content,filename,pathname
0,Prologue: The MissionDisclaimer: All character...,10001898.txt,hp/10001898.txt
1,BlackDisclaimer: I do not own Harry PotterAuth...,10004131.txt,hp/10004131.txt
2,"Chapter 1""I'm pregnant.""""""""Mum please say some...",10004927.txt,hp/10004927.txt
3,"Author's Note: Hey, just so you know, this is ...",10007980.txt,hp/10007980.txt
4,Disclaimer: I do not own Harry Potter and frie...,10010343.txt,hp/10010343.txt


In [187]:
count_vectorizerhp = CountVectorizer(stop_words='english',max_features=100,tokenizer=boring_tokenizer)
Xhp = count_vectorizerhp.fit_transform(fanfics_df['content'])
# print(count_vectorizer.get_feature_names())
pd.DataFrame(Xhp.toarray(), columns=count_vectorizerhp.get_feature_names())

,alway,ani,ask,away,becaus,befor,boy,come,couldn,d,...,voic,wa,walk,wand,want,wasn,way,whi,work,year
0,1,1,3,0,0,2,4,1,0,0,...,2,17,3,3,2,0,1,0,1,0
1,2,1,7,10,1,24,16,5,0,2,...,9,29,17,4,7,1,4,5,1,10
2,0,3,0,0,1,1,2,0,0,0,...,0,24,0,2,4,0,2,1,1,2
3,0,2,9,1,1,5,7,4,1,1,...,1,39,12,5,5,0,8,2,3,22
4,1,1,3,5,5,2,11,5,1,1,...,2,38,2,0,20,6,9,6,0,6
5,0,3,0,6,1,6,4,2,5,8,...,3,73,0,11,5,4,6,2,6,0
6,2,11,21,0,12,12,8,9,2,9,...,2,57,14,0,7,2,5,5,3,5
7,3,5,4,2,3,6,2,10,6,4,...,1,48,4,0,4,2,4,2,1,11
8,3,2,3,1,1,5,3,6,2,0,...,4,43,2,1,7,1,1,1,0,1
9,3,6,11,5,14,24,3,5,3,7,...,7,137,2,3,10,14,10,8,10,8


In [190]:
vectorizer = TfidfVectorizer(use_idf=True, stop_words='english')
X = vectorizer.fit_transform(fanfics_df['content'])

from sklearn.cluster import KMeans

number_of_clusters = 8
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)
print("Term frequency:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Term frequency:
Cluster 0: draco hermione harry malfoy said
Cluster 1: sirius remus james said harry
Cluster 2: fred george hermione ron harry
Cluster 3: lily james sirius said evans
Cluster 4: severus harry snape said lily
Cluster 5: said like just know eyes
Cluster 6: harry said ginny ron hermione
Cluster 7: hermione ron harry ginny said


In [191]:
vectorizer = TfidfVectorizer(use_idf=True, stop_words='english')
X = vectorizer.fit_transform(fanfics_df['content'])

from sklearn.cluster import KMeans

number_of_clusters = 8
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)
print("Term frequency vectorizer:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Term frequency vectorizer:
Cluster 0: james sirius lily said potter
Cluster 1: like said just harry severus
Cluster 2: harry said ron ginny hermione
Cluster 3: draco hermione malfoy harry said
Cluster 4: lily james sirius said evans
Cluster 5: sirius remus james harry said
Cluster 6: scorpius rose albus said al
Cluster 7: hermione ron harry ginny said


#### My hypothesis is that the two categories are defined by age maybe:
* one part is about Harry's age, the story about him and his friends
* the other part is the story about his parents and professors at their age